In [1]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

#from tensorflow.python.client import device_lib
from keras import backend
import tensorflow as tf


C:\Users\Ash Gao\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [2]:
# load training and testing ... 

condition = 'all'

new_train = 'train//' + condition + '//'
new_test = 'test//' + condition + '//'
new_val = 'val//' + condition + '//'

def load_vectors(path):
    files = sorted(os.listdir(path))
    X = np.expand_dims(np.zeros((48, 272)), axis=0)
    y = []
    for npy in files:
        if 'Calm' in npy or 'Other' in npy:
            continue
        
        current = np.load(path+npy)
        X = np.vstack((X, current))
        label = [files.index(npy)]*len(current)       
        y = y + label       
    X = X[1:]
    y = to_categorical(y)    
    print(X.shape)
    print(y.shape)    
    return X, y
    
X_train, y_train = load_vectors(new_train)
try:
    X_val, y_val = load_vectors(new_val)
except:
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
X_test, y_test = load_vectors(new_test)

(41068, 48, 272)
(41068, 6)
(9995, 48, 272)
(9995, 6)


In [6]:
def mil_squared_error(y_true, y_pred):
    return tf.keras.backend.square(tf.keras.backend.max(y_pred) - tf.keras.backend.max(y_true))

adam = tf.keras.optimizers.Adam(learning_rate=1e-5)

def create_cnn(num_layers, n_neurons, n_batch, nbindex, dropout, classes, dense_layers):

    model = Sequential()

    model.add(Convolution1D(filters=nbindex, kernel_size=fillength, activation = 'relu',
                            input_shape=(X_train.shape[1], X_train.shape[2]), kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(filters=nbindex*2, kernel_size=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(filters=nbindex*2, kernel_size=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))

    model.add(Convolution1D(filters=nbindex, kernel_size=fillength, activation = 'relu',
                            kernel_constraint=maxnorm(3)))  
    model.add(MaxPooling1D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(dropout))
    
    model.add(Flatten())
    
    for i in range(0, dense_layers):
        model.add(Dense(n_neurons, activation = 'relu'))
        model.add(Dropout(dropout))
        
    # Here, get the empirical class mean and covariance of training samples

    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=adam,
                  metrics=['accuracy', mil_squared_error])

    return model

In [ ]:
sample_rate = 44100
frame_number = 48
hop_length = 441  # frame size= 2 * hop
segment_length = int(sample_rate * 0.2)  # 0.2
segment_pad = int(sample_rate * 0.02)     # 0.02
overlapping = int(sample_rate * 0.1)   # 0.1

classes = 5
NumofFeaturetoUse = 272
n_neurons = 1024 * 4
dense_layers = 10
num_layers = 3
fillength = 3
nbindex = 1024 * 3
dropout = 0.2
n_batch = 128
n_epoch = 1000

In [8]:
model = create_cnn(num_layers=num_layers, 
                   n_neurons=n_neurons, 
                   n_batch=n_batch, 
                   nbindex=nbindex, 
                   dropout=dropout, 
                   classes=classes, 
                   dense_layers=dense_layers)

In [21]:
print("Fit model on training data")

n_epoch = 100

history = model.fit(
    X_train,
    y_train,
    batch_size=n_batch,
    epochs=n_epoch,
    validation_data=(X_val, y_val), verbose=1
)

Fit model on training data
Epoch 1/100
42/42 [==============================] - 3s 66ms/step - loss: 0.0071 - accuracy: 0.9974 - mil_squared_error: 0.0000e+00 - val_loss: 1.4620 - val_accuracy: 0.7310 - val_mil_squared_error: 8.3235e-14
Epoch 2/100
42/42 [==============================] - 3s 64ms/step - loss: 0.0114 - accuracy: 0.9957 - mil_squared_error: 0.0000e+00 - val_loss: 1.4530 - val_accuracy: 0.7277 - val_mil_squared_error: 9.9476e-14
Epoch 3/100
42/42 [==============================] - 3s 64ms/step - loss: 0.0100 - accuracy: 0.9957 - mil_squared_error: 0.0000e+00 - val_loss: 1.4440 - val_accuracy: 0.7366 - val_mil_squared_error: 1.5936e-13
Epoch 4/100
42/42 [==============================] - 3s 65ms/step - loss: 0.0085 - accuracy: 0.9974 - mil_squared_error: 0.0000e+00 - val_loss: 1.4444 - val_accuracy: 0.7383 - val_mil_squared_error: 1.2079e-13
Epoch 5/100
42/42 [==============================] - 3s 63ms/step - loss: 0.0059 - accuracy: 0.9985 - mil_squared_error: 0.0000e+00 -

In [22]:
#from sklearn.metrics import f1_score

y_preds = [np.argmax(val) for val in model.predict(X_test)]
y_trues = [np.argmax(val) for val in y_test]
acc = accuracy_score(y_trues, y_preds)
f1 = f1_score(y_trues, y_preds, average='weighted')

print(acc)
print(f1)

#model.save('clean_models_acc_' + str(acc)[:6] + '_f1_' + str(f1)[:6] + '_cnn.hdf5') 

0.7410824108241082
0.7384204897566805
